<a href="https://colab.research.google.com/github/nikhil-mathews/MastersPr_Predicting-Human-Pathogen-PPIs-using-Natural-Language-Processing-methods/blob/main/4.Final_model/hv_final_model_three_groups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Each train-test group in the source data are used to train and test the model

In [1]:
import pandas as pd
#Google colab does not have pickle
try:
  import pickle5 as pickle
except:
  !pip install pickle5
  import pickle5 as pickle
import os
import seaborn as sns

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D,Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding,Concatenate
from keras.models import Model,load_model
from sklearn.metrics import roc_auc_score,confusion_matrix,roc_curve, auc
from numpy import random
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.metrics import AUC

import sys
sys.path.insert(0,'/content/drive/MyDrive/ML_Data/')
import functions as f

directory = '/content/drive/MyDrive/ML_Data/'

     |████████████████████████████████| 133kB 14.2MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219265 sha256=ebaa852fffe7b30b24a7047ad0d6c61edcc98bac173f7490a9719f7731a65089
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [2]:
def load(name):
    try:
        with open(directory+''+name+'.pickle', 'rb') as handle:
            return pickle.load(handle)
    except:
        with open(directory+''+name+'.pkl', 'rb') as handle:
            return pickle.load(handle)


def to_HYPPI(df):
  df = df.rename(columns={'Virus':'Yersinia'})
  df["Joined"] = [df.loc[row]['Human']+df.loc[row]['Yersinia'] for row in range(df.shape[0])]
  return df

#load('HVindependent_test_group1')

### Test train Gp1

In [10]:
'''Convert to a format acceptable to the model'''
dftest_1D = to_HYPPI(load('HVindependent_test_group1').reset_index(drop=True))
dftrain_1D = to_HYPPI(load('HVtrain_set_group1').reset_index(drop=True))
dftrain_1D.shape,dftest_1D.shape

((179971, 4), (45078, 4))

In [11]:
# f.save(dftrain_1D,'HV_train_1D')
# f.save(dftest_1D,'HV_test_1D')
df_train = f.load('HV_train_1D')

multiple = df_train.shape[0]//20

"""
not perfect. one datapoint is repeated across each subset, and some are lost.
but considering the overall size, impact of this on result is zero
"""

for i in range(20):
  print(i+1)
  df = df_train.loc[multiple*i:multiple*(i+1)].reset_index(drop=True)
  f.combine_AC(df,5)
  f.save(df,'HV_train_'+str(i)+'')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [4]:
train_5D = f.load('HV_train_0')
for i in range(1,20):
  train_5D = pd.concat([train_5D,f.load('HV_train_'+str(i)+'')]).reset_index(drop=True)
train_5D
f.create_tokenizers(train_5D)

Saved tokenizers as doubleip_tkrs
Saved tokenizer as join_tkr


In [2]:
df_test = f.load('HV_test_1D')
f.combine_AC(df_test,5)
#f.save(df_test,'HV_test_5D')

In [3]:
df_test = f.load('HV_test_5D')
tests = f.preprocess(df_test,saveTokrs=False)
#f.save(tests,'tests')

Preprocessing...


In [4]:
df_test_5D = f.load('HV_test_5D')
val_tests = df_test_5D.loc[:1500]
f.save(val_tests,'val_tests')

In [5]:
EMBEDDING_DIM_5D = 50
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 32
EPOCHS = 1
DROP = 0.5
threshold = 0.918
MAX_SEQUENCE_LENGTH_5D_J = 2000
MAX_SEQUENCE_LENGTH_5D_dIP = 1000
num_words_5D_J = 1000000
num_words_5D = 500000

x1_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
x2_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
x3_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)

x1_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x2_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x3_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x4_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x5_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x6_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)

concatenator = Concatenate(axis=1)
x = concatenator([x1_join.output, x2_join.output, x3_join.output, x1_doubleip.output, x2_doubleip.output, x3_doubleip.output, x4_doubleip.output, x5_doubleip.output, x6_doubleip.output])
x = Dense(256)(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid",name="Final")(x)
model_HV = Model(inputs=[x1_join.input, x2_join.input, x3_join.input, x1_doubleip.input, x2_doubleip.input, x3_doubleip.input, x4_doubleip.input, x5_doubleip.input, x6_doubleip.input], outputs=output)

model_HV.compile(loss='binary_crossentropy', optimizer='adam', metrics=AUC())

#model_HV.save(directory+'model_HV')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


### 1st Epoch

In [6]:
df_test = f.load('val_tests')
val_tests = f.preprocess(df_test,saveTokrs=False)
model_HV = load_model(directory+'model_HV')
names = ['HV_train_'+str(i) for i in range(20)]

for name in names:
  print(name)
  df_train = f.load(name)
  trains = f.preprocess(df_train,saveTokrs=False)
  model_HV.fit(trains, df_train['Label'].values, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(val_tests,df_test['Label'].values))

Preprocessing...
HV_train_0
Preprocessing...
282/282 [==============================] - 306s 1s/step - loss: 0.3063 - auc: 0.6779 - val_loss: 0.2249 - val_auc: 0.8431
HV_train_1
Preprocessing...
282/282 [==============================] - 281s 998ms/step - loss: 0.2396 - auc: 0.7972 - val_loss: 0.2090 - val_auc: 0.8639
HV_train_2
Preprocessing...
282/282 [==============================] - 280s 994ms/step - loss: 0.2174 - auc: 0.8415 - val_loss: 0.2039 - val_auc: 0.8775
HV_train_3
Preprocessing...
282/282 [==============================] - 281s 995ms/step - loss: 0.1943 - auc: 0.8865 - val_loss: 0.1724 - val_auc: 0.9078
HV_train_4
Preprocessing...
282/282 [==============================] - 283s 1s/step - loss: 0.1941 - auc: 0.8806 - val_loss: 0.1818 - val_auc: 0.9065
HV_train_5
Preprocessing...
282/282 [==============================] - 277s 984ms/step - loss: 0.2002 - auc: 0.8872 - val_loss: 0.1743 - val_auc: 0.9056
HV_train_6
Preprocessing...
282/282 [==============================] - 

In [7]:
#model_HV.save(directory+'model_HV')
model_HV = load_model(directory+'model_HV')
df_test = f.load('HV_test_5D')
roc_auc_score(df_test['Label'].values, model_HV.predict(f.load('tests')))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


0.93409074883795

### 2nd Epoch

In [3]:
df_test = f.load('val_tests')
val_tests = f.preprocess(df_test,saveTokrs=False)
model_HV = load_model(directory+'model_HV')
names = ['HV_train_'+str(i) for i in range(20)]

for name in names:
  print(name)
  df_train = f.load(name)
  trains = f.preprocess(df_train,saveTokrs=False)
  model_HV.fit(trains, df_train['Label'].values, epochs=1, batch_size=32, validation_data=(val_tests,df_test['Label'].values))

Preprocessing...
HV_train_0
Preprocessing...
282/282 [==============================] - 248s 804ms/step - loss: 0.1493 - auc: 0.9439 - val_loss: 0.1474 - val_auc: 0.9405
HV_train_1
Preprocessing...
282/282 [==============================] - 223s 792ms/step - loss: 0.1407 - auc: 0.9440 - val_loss: 0.1563 - val_auc: 0.9268
HV_train_2
Preprocessing...
282/282 [==============================] - 224s 793ms/step - loss: 0.1446 - auc: 0.9409 - val_loss: 0.1499 - val_auc: 0.9370
HV_train_3
Preprocessing...
282/282 [==============================] - 222s 786ms/step - loss: 0.1310 - auc: 0.9545 - val_loss: 0.1554 - val_auc: 0.9322
HV_train_4
Preprocessing...
282/282 [==============================] - 221s 783ms/step - loss: 0.1337 - auc: 0.9522 - val_loss: 0.1525 - val_auc: 0.9326
HV_train_5
Preprocessing...
282/282 [==============================] - 222s 786ms/step - loss: 0.1434 - auc: 0.9503 - val_loss: 0.1550 - val_auc: 0.9311
HV_train_6
Preprocessing...
282/282 [============================

In [4]:
#model_HV.save(directory+'model_HV')
model_HV = load_model(directory+'model_HV')
df_test = f.load('HV_test_5D')
roc_auc_score(df_test['Label'].values, model_HV.predict(f.load('tests')))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


0.9383882121542471

### Test train Gp2

In [9]:
dftest_1D = to_HYPPI(load('HVindependent_test_group2').reset_index(drop=True))
dftrain_1D = to_HYPPI(load('HVtrain_set_group2').reset_index(drop=True))
dftrain_1D.shape,dftest_1D.shape

((180342, 4), (45040, 4))

In [7]:
# f.save(dftrain_1D,'HV_train_1D')
# f.save(dftest_1D,'HV_test_1D')
df_train = f.load('HV_train_1D')

multiple = df_train.shape[0]//20

for i in range(20):
  print(i+1)
  df = df_train.loc[multiple*i:multiple*(i+1)].reset_index(drop=True)
  f.combine_AC(df,5)
  f.save(df,'HV_train_'+str(i)+'')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [2]:
train_5D = f.load('HV_train_0')
for i in range(1,20):
  train_5D = pd.concat([train_5D,f.load('HV_train_'+str(i)+'')]).reset_index(drop=True)
train_5D
f.create_tokenizers(train_5D)

Saved tokenizers as doubleip_tkrs
Saved tokenizer as join_tkr


In [2]:
df_test = f.load('HV_test_1D')
f.combine_AC(df_test,5)
f.save(df_test,'HV_test_5D')

In [3]:
df_test = f.load('HV_test_5D')
tests = f.preprocess(df_test,saveTokrs=False)
f.save(tests,'tests')
val_tests = df_test.loc[:1500]
f.save(val_tests,'val_tests')

Preprocessing...


In [4]:
EMBEDDING_DIM_5D = 50
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 32
EPOCHS = 1
DROP = 0.5
threshold = 0.918
MAX_SEQUENCE_LENGTH_5D_J = 2000
MAX_SEQUENCE_LENGTH_5D_dIP = 1000
num_words_5D_J = 1000000
num_words_5D = 500000

x1_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
x2_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
x3_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)

x1_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x2_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x3_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x4_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x5_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x6_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)

concatenator = Concatenate(axis=1)
x = concatenator([x1_join.output, x2_join.output, x3_join.output, x1_doubleip.output, x2_doubleip.output, x3_doubleip.output, x4_doubleip.output, x5_doubleip.output, x6_doubleip.output])
x = Dense(256)(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid",name="Final")(x)
model_HV = Model(inputs=[x1_join.input, x2_join.input, x3_join.input, x1_doubleip.input, x2_doubleip.input, x3_doubleip.input, x4_doubleip.input, x5_doubleip.input, x6_doubleip.input], outputs=output)

model_HV.compile(loss='binary_crossentropy', optimizer='adam', metrics=AUC())

#model_HV.save(directory+'model_HV')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


### 2 epochs

In [5]:
df_test = f.load('val_tests')
val_tests = f.preprocess(df_test,saveTokrs=False)
model_HV = load_model(directory+'model_HV')
names = ['HV_train_'+str(i) for i in range(20)]

for name in names+names:
  print(name)
  df_train = f.load(name)
  trains = f.preprocess(df_train,saveTokrs=False)
  model_HV.fit(trains, df_train['Label'].values, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(val_tests,df_test['Label'].values))

Preprocessing...
HV_train_0
Preprocessing...
282/282 [==============================] - 250s 812ms/step - loss: 0.2955 - auc: 0.6732 - val_loss: 0.2440 - val_auc: 0.8288
HV_train_1
Preprocessing...
282/282 [==============================] - 226s 800ms/step - loss: 0.2368 - auc: 0.8203 - val_loss: 0.2385 - val_auc: 0.8653
HV_train_2
Preprocessing...
282/282 [==============================] - 225s 796ms/step - loss: 0.2236 - auc: 0.8438 - val_loss: 0.2093 - val_auc: 0.8893
HV_train_3
Preprocessing...
282/282 [==============================] - 225s 799ms/step - loss: 0.2059 - auc: 0.8767 - val_loss: 0.1941 - val_auc: 0.8988
HV_train_4
Preprocessing...
282/282 [==============================] - 222s 786ms/step - loss: 0.1977 - auc: 0.8815 - val_loss: 0.1895 - val_auc: 0.9090
HV_train_5
Preprocessing...
282/282 [==============================] - 218s 772ms/step - loss: 0.1977 - auc: 0.8848 - val_loss: 0.1737 - val_auc: 0.9241
HV_train_6
Preprocessing...
282/282 [============================

In [6]:
#model_HV.save(directory+'model_HV')
model_HV = load_model(directory+'model_HV')
df_test = f.load('HV_test_5D')
roc_auc_score(df_test['Label'].values, model_HV.predict(f.load('tests')))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


0.9369066591164074

### Test train Gp3

In [3]:
dftest_1D = to_HYPPI(load('HVindependent_test_group3').reset_index(drop=True))
dftrain_1D = to_HYPPI(load('HVtrain_set_group3').reset_index(drop=True))
dftrain_1D.shape,dftest_1D.shape

((180291, 4), (44962, 4))

In [5]:
# f.save(dftrain_1D,'HV_train_1D')
# f.save(dftest_1D,'HV_test_1D')

df_train = f.load('HV_train_1D')

multiple = df_train.shape[0]//20

for i in range(20):
  print(i+1)
  df = df_train.loc[multiple*i:multiple*(i+1)].reset_index(drop=True)
  f.combine_AC(df,5)
  f.save(df,'HV_train_'+str(i)+'')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [2]:
train_5D = f.load('HV_train_0')
for i in range(1,20):
  train_5D = pd.concat([train_5D,f.load('HV_train_'+str(i)+'')]).reset_index(drop=True)
train_5D
f.create_tokenizers(train_5D)

Saved tokenizers as doubleip_tkrs
Saved tokenizer as join_tkr


In [2]:
df_test = f.load('HV_test_1D')
f.combine_AC(df_test,5)
f.save(df_test,'HV_test_5D')

In [3]:
df_test = f.load('HV_test_5D')
tests = f.preprocess(df_test,saveTokrs=False)
f.save(tests,'tests')
val_tests = df_test.loc[:1500]
f.save(val_tests,'val_tests')

Preprocessing...


In [4]:
EMBEDDING_DIM_5D = 50
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 32
EPOCHS = 1
DROP = 0.5
threshold = 0.918
MAX_SEQUENCE_LENGTH_5D_J = 2000
MAX_SEQUENCE_LENGTH_5D_dIP = 1000
num_words_5D_J = 1000000
num_words_5D = 500000

x1_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
x2_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
x3_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)

x1_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x2_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x3_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x4_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x5_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x6_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)

concatenator = Concatenate(axis=1)
x = concatenator([x1_join.output, x2_join.output, x3_join.output, x1_doubleip.output, x2_doubleip.output, x3_doubleip.output, x4_doubleip.output, x5_doubleip.output, x6_doubleip.output])
x = Dense(256)(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid",name="Final")(x)
model_HV = Model(inputs=[x1_join.input, x2_join.input, x3_join.input, x1_doubleip.input, x2_doubleip.input, x3_doubleip.input, x4_doubleip.input, x5_doubleip.input, x6_doubleip.input], outputs=output)

model_HV.compile(loss='binary_crossentropy', optimizer='adam', metrics=AUC())

#model_HV.save(directory+'model_HV')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


In [3]:
df_test = f.load('val_tests')
val_tests = f.preprocess(df_test,saveTokrs=False)
model_HV = load_model(directory+'model_HV')
names = ['HV_train_'+str(i) for i in range(20)]

for name in names+names:
  print(name)
  df_train = f.load(name)
  trains = f.preprocess(df_train,saveTokrs=False)
  model_HV.fit(trains, df_train['Label'].values, epochs=1, batch_size=32, validation_data=(val_tests,df_test['Label'].values))

Preprocessing...
HV_train_0
Preprocessing...
282/282 [==============================] - 265s 868ms/step - loss: 0.3028 - auc: 0.6716 - val_loss: 0.2539 - val_auc: 0.8342
HV_train_1
Preprocessing...
282/282 [==============================] - 249s 882ms/step - loss: 0.2395 - auc: 0.8218 - val_loss: 0.1959 - val_auc: 0.8711
HV_train_2
Preprocessing...
282/282 [==============================] - 243s 863ms/step - loss: 0.2175 - auc: 0.8507 - val_loss: 0.1984 - val_auc: 0.8936
HV_train_3
Preprocessing...
282/282 [==============================] - 243s 863ms/step - loss: 0.2055 - auc: 0.8639 - val_loss: 0.1950 - val_auc: 0.9069
HV_train_4
Preprocessing...
282/282 [==============================] - 238s 844ms/step - loss: 0.1965 - auc: 0.8833 - val_loss: 0.1719 - val_auc: 0.9157
HV_train_5
Preprocessing...
282/282 [==============================] - 248s 879ms/step - loss: 0.1909 - auc: 0.8921 - val_loss: 0.1679 - val_auc: 0.9169
HV_train_6
Preprocessing...
282/282 [============================

In [4]:
#model_HV.save(directory+'model_HV')
model_HV = load_model(directory+'model_HV')
df_test = f.load('HV_test_5D')
roc_auc_score(df_test['Label'].values, model_HV.predict(f.load('tests')))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


0.9427204120284928